<a href="https://colab.research.google.com/github/Filipriec/zaklady_pythonu/blob/main/10_datove_struktury2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dátové štruktúry 2
## Array (dynamické pole)

### Čo je dynamické pole?

Dynamické pole (v Pythone `list`) je štruktúra, ktorá umožňuje:
- **Rýchly prístup k prvkom podľa indexu** (O(1))
- **Flexibilnú veľkosť** (automaticky rastie pri pridávaní)
  
Napriek názvu *pole* je v Pythone implementované ako **dynamické pole s over-allocation** (pri plnení sa kapacita zdvojnásobí(približne)).

---


### Základné operácie a časová zložitosť

| Operácia           | Časová zložitosť | Poznámka                      |
|--------------------|------------------|-------------------------------|
| `a[i]`             | O(1)             | Indexovaný prístup            |
| `a.append(x)`      | O(1) amortizované | Pridanie na koniec            |
| `a.insert(i, x)`   | O(n)             | Pomalé – posúva prvky         |
| `a.pop()`          | O(1)             | Odstránenie z konca           |
| `a.pop(0)`         | O(n)             | Odstránenie zo začiatku       |
| `a.remove(x)`      | O(n)             | Hľadanie + posun prvkov       |
| `x in a`           | O(n)             | Lineárne vyhľadávanie         |

---

## Praktické využitie Python `list`

### 1. Efektívne operácie


In [ ]:
# Správne: O(1) operácie
data = []
data.append(10)      # Rýchle
data.append(20)
last = data.pop()    # Rýchle
print(data)

In [ ]:
# Pomalé: O(n) operácie
data = [1, 2, 3, 4, 5]
data.insert(0, 0)    # Pomalé pre veľké polia - musí posunúť všetko
print(data)
data.pop(0)          # Pomalé - musí posunúť všetko doľava
print(data)

### Prečo je `insert(0)` a `pop(0)` pomalé?

Pri vložení na začiatok sa musia **všetky prvky posunúť doprava**:

```
insert(0, "X"):

Pred:   ┌───┬───┬───┬───┐
        │ a │ b │ c │ d │
        └───┴───┴───┴───┘
          ↓   ↓   ↓   ↓     (posun všetkých prvkov)
Po:     ┌───┬───┬───┬───┬───┐
        │ X │ a │ b │ c │ d │
        └───┴───┴───┴───┴───┘
```

Čím viac prvkov, tým viac práce → O(n).

---



### 2. Rast kapacity
Python `list` alokuje viac miesta v pamäti, ako je potrebné, aby sa vyhol častým realokáciám:

In [ ]:
import sys

lst = []
predchadzajuca = 0

for i in range(20):
    velkost = sys.getsizeof(lst)
    if velkost > predchadzajuca:
        print(f"Prvkov: {len(lst):2}, Veľkosť: {velkost} bajtov ← REALOKÁCIA")
    else:
        print(f"Prvkov: {len(lst):2}, Veľkosť: {velkost} bajtov")
    predchadzajuca = velkost
    lst.append(i)

## Kedy nepoužiť `list`?

### Prípad 1: Časté operácie na začiatku

In [ ]:
from collections import deque

# Zlé pre list, dobré pre deque
items = deque(maxlen=1000)  # Kruhový buffer
items.appendleft(10)        # O(1)
items.popleft()            # O(1)

## Kedy nepoužiť `list`?

### Časté operácie na začiatku listu → použite `deque`

In [ ]:
from collections import deque

# deque má O(1) operácie na OBOCH koncoch
d = deque([1, 2, 3])

d.appendleft(0)   # O(1)
print(d)
d.popleft()       # O(1)
print(d)


## Benchmark: `list` vs `deque`

In [ ]:
from collections import deque
import time

def zmeraj(operacia, popis):
    start = time.perf_counter()
    operacia()
    koniec = time.perf_counter()
    print(f"{popis}: {(koniec - start)*1000:.2f} ms")

n = 50_000

print(f"Porovnanie {n:,} operácií na ZAČIATKU:\n")

def test_list():
    lst = []
    for i in range(n):
        lst.insert(0, i)

def test_deque():
    d = deque()
    for i in range(n):
        d.appendleft(i)

zmeraj(test_list, "list.insert(0)")
zmeraj(test_deque, "deque.appendleft()")

---

## Porovnanie `list` vs `deque`

| Operácia | List | Deque | Optimálnejšie? |
|----------|------|-------|-------------|
| `a[i]` prístup | O(1) | O(n) | **List** |
| Pridať na koniec | O(1) | O(1) | Rovnaké |
| Pridať na začiatok | O(n) | O(1) | **Deque** |
| Odobrať z konca | O(1) | O(1) | Rovnaké |
| Odobrať zo začiatku | O(n) | O(1) | **Deque** |
| Hľadať `x in a` | O(n) | O(n) | Rovnaké |

---

## Deque (double-ended queue)

### Čo je deque?

Deque je dátová štruktúra s rýchlymi operáciami na oboch koncoch. V Pythone je implementovaný ako **double linked list** (obojsmerne spájaný zoznam):
```
       ┌───────────┐     ┌───────────┐     ┌───────────┐
       │ ● │ A │ ● │ ←─→ │ ● │ B │ ● │ ←─→ │ ● │ C │ ● │
       └───────────┘     └───────────┘     └───────────┘
         ↓                                           ↓
       None                                        None
```

Každý prvok má odkaz (●) na predchádzajúci aj nasledujúci prvok. Preto vieme pridávať a odoberať z oboch strán bez posúvania ostatných prvkov.

---

### Základné operácie


In [ ]:
from collections import deque

d = deque([1, 2, 3])

# Operácie na konci (ako list)
d.append(4)       # [1, 2, 3, 4]
d.pop()           # [1, 2, 3]

# Operácie na začiatku (toto list nevie efektívne)
d.appendleft(0)   # [0, 1, 2, 3]
d.popleft()       # [1, 2, 3]


### Prečo je deque rýchly na oboch koncoch?

**List (dynamické pole):** Prvky sú uložené za sebou v pamäti. Pri `pop(0)` sa musia všetky posunúť.

**Deque (doubly linked list):** Prvky sú prepojené odkazmi. Pri odobraní len "prepojíme" susedov.
```
Odstránenie zo začiatku deque - O(1):

Pred:   [●│A│●] ←─→ [●│B│●] ←─→ [●│C│●]

Po:                 [●│B│●] ←─→ [●│C│●]

Len sme zmenili jeden pointer - žiadne posúvanie.
```

---

### Nevýhoda deque

Prístup podľa indexu je pomalý, pretože deque nemá priamy prístup k prvkom - musí "prejsť" odkazmi od začiatku alebo konca.

In [ ]:

d = deque([1, 2, 3, 4, 5])
print(d[2])  # O(n) - musí prejsť od začiatku!